In [1]:
import warnings
warnings.filterwarnings("ignore")

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from tqdm import tqdm
# !pip install downcast
import numpy
import time
from downcast import reduce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open(r'/content/drive/MyDrive/Project Files/train_CV_test_data_x_y_id.pkl', 'rb') as f:
    x_train,x_CV,x_test,y_train,y_CV,y_test,id= pickle.load(f)

In [ ]:
clf=LGBMRegressor(objective ='poisson',learning_rate=0.2,n_estimators=100)
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE on LGBM is:',RMSE)

RMSE on LGBM is: 2.003571591469768


In [ ]:
with open(r'best_LGBM_clf_objective_poision_m5.pkl','wb') as f:
    pickle.dump([clf],f)

In [ ]:
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/best_LGBM_clf_objective_poision_m5.pkl'
target=r'/content/drive/MyDrive/Project Files/best_LGBM_clf_objective_poision_m5.pkl'
shutil.copyfile(original,target)

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "shantanuekhande"
os.environ['KAGGLE_KEY'] = "7c0f27a1e48312dc37124571a15b336e"
!kaggle competitions download -c 'm5-forecasting-accuracy'  -p 'train'    ## getting all data into train folder 

 58% 9.00M/15.5M [00:00<00:00, 20.7MB/s]
100% 15.5M/15.5M [00:00<00:00, 31.6MB/s]
 84% 12.0M/14.2M [00:00<00:00, 33.6MB/s]
100% 14.2M/14.2M [00:00<00:00, 47.4MB/s]
 69% 11.0M/15.8M [00:00<00:00, 28.1MB/s]
100% 15.8M/15.8M [00:00<00:00, 44.9MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 112MB/s]
  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 156MB/s]


In [ ]:
! mkdir final_train

In [ ]:
!7z e /content/train/sales_train_evaluation.csv.zip -o/content/final_train
!7z e /content/train/sales_train_validation.csv.zip -o/content/final_train
!7z e /content/train/sample_submission.csv.zip -o/content/final_train
!7z e /content/train/calendar.csv.zip -o/content/final_train
!7z e /content/train/sell_prices.csv.zip -o/content/final_train


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/train/                         1 file, 16611637 bytes (16 MiB)

Extracting archive: /content/train/sales_train_evaluation.csv.zip
--
Path = /content/train/sales_train_evaluation.csv.zip
Type = zip
Physical Size = 16611637

  0%     35% - sales_train_evaluation.csv                                  73% - sales_train_evaluation.csv                                 Everything is Ok

Size:       121736518
Compressed: 16611637

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs AMD EP

In [42]:
sales = pd.read_csv(r'/content/final_train/sales_train_evaluation.csv')
# sales.name = 'sales'
calendar = pd.read_csv(r'/content/train/calendar.csv')
# calendar.name = 'calendar'
prices = pd.read_csv(r'/content/final_train/sell_prices.csv')
# prices.name = 'prices'

In [7]:
# Reducing memory size of DataFrame using Reduce function from downcasting library.
# https://medium.com/@deepakec1031/how-to-reduce-pandas-data-frame-size-using-downcast-e00cfed8e2c5
sales= reduce(sales)
prices= reduce(prices)
calendar= reduce(calendar)

In [ ]:
sales.to_csv('sales.csv', index=False)

In [ ]:
calendar.to_csv('calendar.csv', index=False)

In [ ]:
a=pd.read_csv(r'prices.csv')
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


### M5 compitation is sales forcasting compitation in which data taken from walmart.in that we have in total 3049 unique product present. so my task is to predict next 28 days sales for any particular product product.so i am writing two function the first function will take the product as input and return next 28 days for that product.(d_1914 to d_1942). the second function will take product and ground truth value as input and return RMSE score as output.

---



In [61]:
def function_1(x):
  ## gathering  data into single dataframe  
  x=sales[sales['id']==x]
  df_sales=pd.melt(x, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

  # filling NaN values with no_event from calender features (event_name_1,event_type_1,event_name_2,event_type_ )
  calendar.fillna('No_event',inplace=True)
  df_final= pd.merge(df_sales, calendar, on='d', how='left')
  df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

  ## filling NaN value with zero for sell_price column from df_final
  df_final.fillna(0,inplace=True)

  ## now its time to reduce memory size of df_final dataframe using downcasting technique 
  df_final=reduce(df_final)


  ## now its time to do feature engineering 
  # 1. Time based feature
  # 1.1 Feature name: is_high_sale_months
  f=lambda x: 1 if x==2 or x==3 or x==4 or x==5 else 0
  df_final['is_high_sale_months']=df_final['month'].map(f) 

  # 1.2 Feature name: is_high_sale_day
  f=lambda x: 1 if x==7 or x==1 or x==2 else 0
  df_final['is_high_sale_day']=df_final['wday'].map(f) 

  # 1.3 Feature name:is_christmas
  f=lambda x: 1 if x=='Christmas' else 0
  df_final['is_christmas']=df_final['event_name_1'].map(f)

  ## 2. Lags
  # Feature Name : ['lag_1','lag_7', 'lag_14','lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56', 'lag_63','lag_70']
  lags=[1,7,14,21,28,35,42,49,56,63,70]
  for i in lags:
      df_final['lag_'+str(i)]=df_final.groupby(['id'])['sold'].shift(i)

  # if you see in dataframe there is lot of NaN value got created. i am replacing NaN value with "0".
  lags=['lag_1','lag_7', 'lag_14','lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56', 'lag_63','lag_70']
  for i in lags:
      df_final[i]=df_final[i].fillna(0)

  ## 3. Rolling Window
  window=[7,14,21,28,35,42]
  for i in  window:
      df_final['rolling_mean_'+str(i)]=df_final.groupby(['id'])['sold'].transform(lambda s: s.rolling(i).mean())

  # if you see in dataframe there is lot of NaN value got created. i am replacing NaN value with "0".
  window=['rolling_mean_7','rolling_mean_14','rolling_mean_21','rolling_mean_28', 'rolling_mean_35','rolling_mean_42']
  for i in window:
      df_final[i]=df_final[i].fillna(0)

  ## 4. Expanding Window
  df_final['expanding_mean']=df_final.groupby(['id'])['sold'].transform(lambda s: s.expanding().mean())

  # if you see in dataframe there is lot of NaN value got created. i am replacing NaN value with "0".
  df_final['expanding_mean']=df_final['expanding_mean'].fillna(0)

  ## 5. Label Encoding:in this we are going to convert categorical variable into numeric using sklearn preprocessing library called LabelEncoder
  labelencoder=LabelEncoder() 
  category=['event_name_1','event_type_1','event_name_2','event_type_2','item_id','dept_id','cat_id','store_id','state_id']
  for i in category:
      df_final[i+'_']=labelencoder.fit_transform(df_final[i])

  # droping duplicate fetures 
  df_final=df_final.drop(['event_name_1','event_type_1','event_name_2','event_type_2','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

  # removing hypen from "d" features : d_1-----> 1, d_100----->100
  l=[]
  for i in df_final['d']:
      l.append(i.split('_')[1])
  df_final['day']=l
  df_final['day']=df_final['day'].astype(np.int16)


  # dropping unusefull feature
  df_final=df_final.drop(['event_name_1_','event_name_2_','item_id_','d','date','wm_yr_wk','weekday','year'],axis = 1)
  


  ## storing id
  id=df_final[['id','day']]

  ## now dividing data into input and output form
  x=df_final.drop(columns=['id','sold'])
  y=df_final[['day','sold']]

  x_test=x.loc[x['day']>=1914]
  x_test=x_test.drop(columns=['day'])

  y_test=y.loc[y['day']>=1914]
  y_test=y_test.drop(columns=['day'])

  ## applying LGBM regressor 
  # with open(r'/content/drive/MyDrive/Project Files/best_LGBM_clf_objective_poision_m5.pkl', 'rb') as f:
  #   clf= pickle.load(f)
  y_predicted=clf[0].predict(x_test)

  ## converting data into submission format 
  predicted_=pd.DataFrame({'sold_predicted':y_predicted})
  predicted_=predicted_.apply(lambda x :round(x))
  predicted_= predicted_['sold_predicted'].astype('int16')
  
  id_index=id.reset_index()
  id_index.drop(columns='index',inplace=True)


  final_format=pd.merge(id_index, predicted_, left_index=True, right_index=True)
  final_format.pivot(index=['id'], columns='day')


  final_LGBM= final_format.pivot(index='id', columns='day')['sold_predicted'].reset_index()
  final_LGBM.columns.name = None


  return(final_LGBM,y_test,y_predicted)

In [62]:
def function_2(y_test,y_predicted):
  ## calculating RMSE 
  RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
  print('RMSE using LGBMRegressor:',RMSE)

In [28]:
x=sales[sales['id']=='FOODS_3_384_CA_4_evaluation']
x

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
11755,FOODS_3_384_CA_4_evaluation,FOODS_3_384,FOODS_3,FOODS,CA_4,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,1,2,0,2,1,0,0,2,2,2,1,0,2,0,1,4,0,1,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,0


In [63]:
import time
start=time.clock() 
final_LGBM,y_test,y_predicted=function_1('FOODS_3_384_CA_4_evaluation')
end=time.clock()
total_time=end - start
print('total time required to predict next 28 days sale for one product is :',total_time)
final_LGBM

total time required to predict next 28 days sale for one product is : 1.393987999999922


,id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,FOODS_3_384_CA_4_evaluation,0,3,0,1,0,0,0,2,1,0,0,0,0,0,1,1,2,1,0,0,0,0,2,4,0,0,0,0


In [56]:
start=time.clock() 
function_2(y_test,y_predicted)
end=time.clock()
total_time=end - start
print('total time required to RMSE score for one product is :',total_time)

RMSE using LGBMRegressor: 1.5081875192553258
total time required to RMSE score for one product is : 0.00153499999998985
